## Student: Rodolfo Lerma

# Introduction to TD, SARSA, and Q-Learning (Part 1)

## Machine Learning 530

## Stephen Elston


In the previous lesson we explored Monte Carlo **reinforcement learning**. MC RL required that the returns for an entire episode be computed before any values are available for use. The disadvantage of this approach is the the full set of returns are required for state value or action value estimates. But, how can we get state values or action-values in fewer time steps? It turns out there are algorithms which compute estimates in as few as one step. In this lesson will focus on i) a state-value estimation algorithm known as **time difference learning**, ii) **TD-learning** and control algorithm known as **SARSA** and an **off-policy** algorithm known as **Q-Learning**. 

Recall that reinforcement learning has several distinctive characteristics, which differentiate this method from other machine learning and dynamic programming:
- **No Markov model** needs to be specified for reinforcement learning, in contrast to dynamic programming.
- Like dynamic programming, reinforcement learning **optimizes a reward function**. This is in contrast to supervised and unsupervised learning which use an error or objective function.  
- Reinforcement learning algorithms learn by **experience**. Over time, the algorithm learns a model of the environment and these results are used to optimize the expected reward. Learning from experience is in contrast to supervised learning which uses known marked cases. 
- Reinforcement learning agents take **actions** and only receive **state** and **rewards** from the environment. These are the only interaction between the RL agent and the environment.    

The interaction between a reinforcement learning agent and the environment are illustrated in the figure below. Notice that the only feedback the agent receives from the environment is reward and state.   

<img src="img/RL_AgentModel.JPG" alt="Drawing" style="width:500px; height:300px"/>
<center> **Reinforcement Learning Agent and Environment** </center>  

The ability to learn from experience is an attractive concept. This method of learning seems to mimic human learning. However, reinforcement learning has proven difficult to use in real-world applications. For a review of successes and problems arising when applying RL to robotics see [Kobler et. al.](https://www.ias.informatik.tu-darmstadt.de/uploads/Publications/Kober_IJRR_2013.pdf). At the present time, RL has mostly succeeded in cases where simulations can be used to gain experience. 

**Suggested readings** for TD and Q reinforcement learning, Chapters 6 and 7 of Sutton and Barto, second edition, provides a good introductions, including many alternative algorithms and details not discussed here.   

## TD Prediction Model

In a previous lesson we examined a general update model:

$$NewValue = OldValue + LearningRate * ErrorTerm$$

Following this general formulation the update for TD state value can be written as:

$$V_{t+1}(S_{t}) = V_t(S_t) + \alpha \big[ G_t - V_t(S_t) \big]$$

Here,   
$\alpha = $ the learning rate,      
$\big[ G_t - V_t(S_t) \big] = $ the TD error term is the difference between the return $G_t$ and the value, $V(S_t)$. The return is identical to the state value at convergence, making the error 0.   

   
Using the same general formulation we can create and single time step update model know as the **one step time difference** or **TD(0)** algorithm.

$$V_{t+1}(S_{t}) = V_t(S_t) + \alpha \big[ R_{t+1} + V_{t+1}(S_{t+1}) - V_t(S_t) \big]$$

Where,  
$\delta_t =  R_{t+1} + V_{t+1}(S_{t+1}) - V_t(S_t) = $ the one-step **TD error**,  
$R_{t+1} = $ the return for the next time step,   
$V_t(S_t) = $ is the state-value at time step t,   
$V_{t+1}(S_{t+1}) = $ the bootstrap state-value for the successor state, $S_{t+1}$.

Like dynamic programming algorithms, the TD algorithm **bootstraps**. The return and estimated value at the next time step, $V_{t+1}(S_{t+1})$, are from previous samples. However, unlike MC RL which does not bootstrap, the TD(0) algorithm produces an estimate of $V_{t+1}(S_t)$ in only one time step, rather than waiting to reach the terminal state at the end of the episode. This fact can be seen by examining the backup diagram shown below:

<img src="img/TD0.JPG" alt="Drawing" style="width:60px; height:150px"/>
<center> **Backup Diagram of TD(0)** </center>

## Example of Time Difference RL

With this short introduction TD RL in mind, let's try an example. We will sample the value function using a basic TD(0) algorithm here. 

As discussed in other labs, **Navigation** to a goal is a significant problem in robotics. Real-world navigation is rather complex. Therefore, in this example we will use a simple analog called a **grid world**. The grid world for this problem is shown below. 

<img src="img/GridWorld.JPG" alt="Drawing" style="width:200px; height:200px"/>
<center> **A 4x4 Grid World with Terminal State** </center>

The grid world consists of a 4x4 set of positions the robot can occupy. Each position is considered a state. The goal is to navigate to state 0, the goal, in the minimum steps. We will explore methods to find policies which reach this goal and achieve maximum reward. 

Grid position 0 is the goal and a **terminal state**. There are no possible state transitions out of this position. The presence of a terminal state makes this an **episodic Markov random process**. For each episode sampled the robot can start in any other random position, $\{ 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15 \}$. This random selection process makes this a **random start** TD algorithm. The episode terminates when the robot enters the terminal position (state 0).  

In reality, an RL agent may need to explore to find the possible actions when it is in some particular state. To simplify our example, we encode, or represent, these possibilities in a dictionary as shown in the code block below. We use a dictionary of dictionaries to perform the lookup. The keys of the outer dictionary are the identifiers (numbers) of the states. The keys of the inner dictionary are the possible actions and the values are the **successor state**, $s'$, for that transition.  

In each state, there are four possible actions the robot can take:
- up, u
- down, d,
- left, l
- right, r

The TD RL agent has no model for the environment. Therefore, beyond these allowed actions, all other information is encapsulated in the environment and is unobservable by the agent. **This is the key difference between reinforcement learning and dynamic programming.** 

In [1]:
## import numpy for latter
import numpy as np
import numpy.random as nr
import pandas as pd

## Define the transition dictonary of dictionaries:
neighbors = {0:{'u':0, 'd':0, 'l':0, 'r':0},
          1:{'u':1, 'd':5, 'l':0, 'r':2},
          2:{'u':2, 'd':6, 'l':1, 'r':3},
          3:{'u':3, 'd':7, 'l':2, 'r':3},
          4:{'u':0, 'd':8, 'l':4, 'r':5},
          5:{'u':1, 'd':9, 'l':4, 'r':6},
          6:{'u':2, 'd':10, 'l':5, 'r':7},
          7:{'u':3, 'd':11, 'l':6, 'r':7},
          8:{'u':4, 'd':12, 'l':8, 'r':9},
          9:{'u':5, 'd':13, 'l':8, 'r':10},
          10:{'u':6, 'd':14, 'l':9, 'r':11},
          11:{'u':7, 'd':15, 'l':10, 'r':11},
          12:{'u':8, 'd':12, 'l':12, 'r':13},
          13:{'u':9, 'd':13, 'l':12, 'r':14},
          14:{'u':10, 'd':14, 'l':13, 'r':15},
          15:{'u':11, 'd':15, 'l':14, 'r':15}}

To simulate the environment, we need a reward structure. In this case, the robot receives the following rewards:   

- 10 for entering position 0. 
- -1 for attempting to leave the grid. In other words, we penalize the robot for hitting the edges of the grid.  
- -0.1 for all other state transitions, which is the cost for the robot to move from one state to another. If we did not have this penalty, the robot could follow any random plan to the goal which did not hit the edges. 

This **reward structure is unknown to the TD RL agent**. The agent must **learn** the rewards by sampling the environment. Here the rewards are in the form of action values.    

We encode these rewards in the same type of dictionary structure used for the foregoing structures. 

In [2]:
rewards = {0:{'u':10.0, 'd':10.0, 'l':10.0, 'r':10.0},
          1:{'u':-1, 'd':-0.1, 'l':10.0, 'r':-0.1},
          2:{'u':-1.0, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          3:{'u':-1.0, 'd':-0.1, 'l':-0.1, 'r':-1.0},
          4:{'u':10.0, 'd':-0.1, 'l':-1.0, 'r':-0.1},
          5:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          6:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          7:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-1.0},
          8:{'u':-0.1, 'd':-0.1, 'l':-1.0, 'r':-0.1},
          9:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          10:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          11:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-1.0},
          12:{'u':-0.1, 'd':-1.0, 'l':-1.0, 'r':-0.1},
          13:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-0.1},
          14:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-0.1},
          15:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-1.0}}

As was done previously, we will use an environment simulator. The function is called with a state and action. It returns the next state, 's_prime' and 'reward'. 

To simplify the rest of the code in this notebook we are treating the dictionaries as global. In general, this would be considered poor programming practice. 

Execute the code in the cell below and observe the results from the test cases. 

In [3]:
def simulate_environment(s, action, neighbors = neighbors, rewards = rewards, terminal = 0):
    """
    Function simulates the environment
    returns s_prime and reward given s and action
    """
    s_prime = neighbors[s][action]
    reward = rewards[s][action]
    return (s_prime, reward, is_terminal(s_prime, terminal))

def is_terminal(state, terminal = 0):
    return state == terminal

In [4]:
## Test the function
for a in ['u', 'd', 'r', 'l']:
    print(simulate_environment(1, a))

(1, -1, False)
(5, -0.1, False)
(2, -0.1, False)
(0, 10.0, True)


### TD(0) Policy Evaluation

We have everything in place to perform TD(0) policy evaluation for the grid world. The code in the cell below implements the TD(0) algorithm and applies it to the policy in the grid world. Notice that the algorithm makes calls to the aforementioned `state_values` function to find information on successor state and rewards for a transition given a current state and action. Execute this code and examine the results. 

In [5]:
initial_policy = {0:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        1:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25}, 
                        2:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        3:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        4:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        5:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        6:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        7:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        8:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        9:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        10:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        11:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        12:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        13:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        14:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        15:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25}}

The code in the cell below performs a random start for the beginning of an episode. The random start cannot be in the terminal state. 

In [6]:
def start_episode(n_states):
    '''Function to find a random starting value for the episode
    that is not the terminal state'''
    state = nr.choice(range(n_states))
    while(is_terminal(state)):
         state = nr.choice(range(n_states))
    return state

In [7]:
## test the function to make sure never starting in terminal state
[start_episode(15) for _ in range(10)]

[8, 7, 8, 8, 1, 5, 2, 2, 14, 4]

The function in the cell below finds and action given the state. The probability of the action is determined by the policy. Given the action the next state, reward and a terminal flag are found.     

In [8]:
def take_action(state, policy, actions = {1:'u', 2:'d', 3:'l', 4:'r'}):
    '''Function takes action given state using the transition probabilities 
    of the policy'''
    ## Find the action given the transistion probabilities defined by the policy.
    action = actions[nr.choice(range(len(actions)), p = list(policy[state].values())) + 1]
    s_prime, reward, terminal = simulate_environment(state, action)
    return (action, s_prime, reward, terminal)

In [9]:
## Test function for several states
for s in range(16):
    print(take_action(s, initial_policy))

('u', 0, 10.0, True)
('d', 5, -0.1, False)
('r', 3, -0.1, False)
('u', 3, -1.0, False)
('r', 5, -0.1, False)
('d', 9, -0.1, False)
('u', 2, -0.1, False)
('u', 3, -0.1, False)
('r', 9, -0.1, False)
('r', 10, -0.1, False)
('d', 14, -0.1, False)
('l', 10, -0.1, False)
('l', 12, -1.0, False)
('r', 14, -0.1, False)
('d', 14, -1.0, False)
('u', 11, -0.1, False)


> **Exercise 9-1-1:** The function in the cell below computes the **state-values** using the one-step TD(0) algorithm. The loop in the function iterates over the samples to update the state-values. You can see additional details by reading the code comments. Now, do the following:   
> 1. Complete the missing line of code which computes the TD(0) error and execute your code.  
> 2. As a basic test of the state value estimation function, examine the results. Do these state values have a reasonable relationship to the goal and each other and why?  

In [10]:
def td_0_state_values(policy, n_samps, alpha = 0.05, gamma = 1.0):
    """
    Function for TD(0) policy evalutation
    """
    ## Find the starting state
    n_states = len(policy)
    current_state = start_episode(n_states)
    terminal = False
    #2.
    ## Array for state values
    v = np.zeros((n_states,1))
    
    for _ in range(n_samps):
        ## Find the next action and reward
        action, s_prime, reward, terminal = take_action(current_state, policy)
        
        ######### Complete the code below ##############
        ## Compute the TD(0) error
        delta = reward + gamma*v[s_prime] - v[current_state]

        ## Update the state value
        v[current_state] = v[current_state] + alpha*delta
        current_state = s_prime
        if(terminal): ## start new episode when terminal
            current_state = start_episode(n_states)
    return(v)

In [11]:
state_values = td_0_state_values(initial_policy, 20000).reshape((4,4))        
print(np.sum(state_values))
print(state_values)

-42.563412603182925
[[ 0.          2.54359911 -1.55417655 -4.77145706]
 [ 4.03728659  0.14330501 -3.55463168 -5.03974969]
 [-0.70919025 -2.822326   -4.52547045 -5.66963232]
 [-3.71463024 -4.47068952 -5.5115714  -6.94407816]]


**Answer:**    
> 2. As a basic test of the state value estimation function, examine the results. Do these state values have a reasonable relationship to the goal and each other and why?  
The state values seem reasonable as the smaller values (more negative numbers) are the ones further away.

## One Step SARSA Algorithm

Now that we have examined the one step TD(0) algorithm for policy (value) evaluation, we need to define a one step algorithm for **control** or **policy improvement**.     

The **state action reward state action** or **SARSA** algorithm is an **on policy** method which uses time differencing to evaluate action values. As you might imagine from the name, this algorithm starts with an **on policy** action from the current state which results in a state transition and a reward. The backup diagram for one step SARSA (SARSA(0)) is shown in the figure below. 

<img src="img/SARSA.JPG" alt="Drawing" style="width:75px; height:150px"/>
<center> **Backup Diagram for SARSA(0)** </center>

Compare the backup diagram for SARSA(0) to the one for TD(0) notice that the order of state and action are reversed between the two algorithms. This realization is a good way to understand the difference. 

The update equation for SARSA(0) is as follows:

$$Q_{t+1}(S_{t},A_{t}) = Q_{t}(S_t,A_t) + \alpha \big[ R_{t+1} + \gamma Q_{t}(S_{t+1},A_{t+1}) - Q_t(S_t,A_t) \big]$$  

Where,   
$Q_t(S_t,A_t) = $ is the action value in state S given action A at step t,   
$R_{t+1} = $ is the reward for the next time step,    
$\delta_t = R_{t+1} + \gamma Q_{t}(S_{t+1},A_{t+1}) - Q_t(S_t,A_t) = $ TD error,   
$Q_{t}(S_{t+1},A_{t+1}) = $ action-value of successor action, $A_{t+1}'$, from the successor state, $S_{t+1}$,   
$\alpha = $ the learning rate,   
$\gamma = $ discount factor.  

> **Exercise 9-1-2:** The code in the cell below implements the SARSA(0) action value algorithm. Complete the code inn the cell below to compute the update of the action value using the above formula. Action values are in the numpy array `q`, indexed by state and action. Then execute your code.     

In [12]:
def print_Q(Q):
    Q = pd.DataFrame(Q, columns = ['up', 'down', 'left', 'right'])
    print(Q)

def new_episode(n_states, policy):
    '''This function provides a start for a TD
    episode making sure the first transition is not 
    the termnal state'''
    current_state = start_episode(n_states)
    ## Find fist action and reward
    action, s_prime, reward, terminal = take_action(current_state, policy)
    return(current_state, action, s_prime, reward, terminal)    

def SARSA_0(policy, n_samps, alpha = 0.02, gamma = 1.0, action_index = {'u':0, 'd':1, 'l':2, 'r':3}):
    """
    Function for TD(0) policy evalutation
    """
    ## Find the starting state
    n_states = len(policy)
    current_state, action, s_prime, reward, terminal = new_episode(n_states, policy)
    action_idx = action_index[action]
    
    ## Array for state values
    q = np.zeros((n_states, len(policy[0])))
    
    for _ in range(n_samps):
        ## Find the next action and reward
        action_prime, s_prime_prime, reward_prime, terminal_prime = take_action(s_prime, policy)
        action_idx_prime = action_index[action_prime]
        
        ######### Complete the code below ##############
        ## Compute the TD error given the action values `q`, the successor state, successor action, 
        ## current state, and current action
        delta = reward + gamma*q[s_prime, action_idx_prime] - q[current_state, action_idx]
        
        ## Update the action values
        q[current_state, action_idx] = q[current_state, action_idx] + alpha*delta
        ## Update the state, action and reward for the next time step
        current_state = s_prime
        s_prime = s_prime_prime
        action = action_prime
        reward = reward_prime
        terminal = terminal_prime
        action_idx = action_idx_prime

        ## Check if end of episode
        if(terminal): 
            ## start new episode
            current_state, action, s_prime, reward, terminal = new_episode(n_states, policy)        
    return(q)

In [13]:
Q = SARSA_0(initial_policy, 20000, alpha = 0.2, gamma = 0.99)
print_Q(Q)

           up       down       left      right
0    0.000000   0.000000   0.000000   0.000000
1   -8.499595  -8.890084  -1.435694 -10.349844
2  -10.732521 -10.187094  -6.931464 -11.053622
3  -12.105536 -11.038268  -9.794738 -12.083858
4   -2.245406  -9.986057  -7.326030  -8.349111
5   -8.209361  -9.549731  -5.210737 -10.165225
6  -10.144006 -10.282686  -8.644856 -10.857459
7  -11.371036 -11.224400  -9.986467 -11.788054
8   -6.308748 -10.961649 -11.464550 -10.370800
9   -9.261018 -10.996236 -10.174024 -10.318588
10  -9.574495 -10.982719 -10.310010 -11.220758
11 -11.077975 -11.284624 -10.011400 -11.906454
12  -9.560522 -11.998994 -12.429078 -11.291366
13 -10.255210 -11.661191 -11.363686 -11.392196
14 -10.393601 -11.678857 -10.983808 -11.612150
15 -11.446858 -12.269117 -10.890251 -12.614561


The array printed above display the action values for each state. 

The function below takes the action values and finds an improved policy. Execute this code and examine the results. 

In [14]:
def action_lookup(index):
    """Helper function returns action given an index"""
    action_dic = {0:'u', 1:'d', 2:'l', 3:'r'}
    return action_dic[index]

def index_lookup(action):
    """Helper function returns index given action"""
    index_dic = {'u':0, 'd':1, 'l':2, 'r':3}
    return index_dic[action]

def update_policy(policy, Q, epsilon):
    '''Updates the policy based on estiamtes of Q using 
    an epslion greedy algorithm. The action with the highest
    action value is used.'''
    
    ## Find the keys for the actions in the policy
    keys = list(policy[0].keys())
    
    ## Iterate over the states and find the maximm action value.
    for state in range(len(policy)):
        ## First find the index of the max Q values  
        q = Q[state,:]
        max_action_index = np.where(q == max(q))[0]

        ## Find the probabilities for the transitions
        n_transitions = float(len(q))
        n_max_transitions = float(len(max_action_index))
        p_max_transitions = (1.0 - epsilon *(n_transitions - n_max_transitions))/(n_max_transitions)
  
        ## Now assign the probabilities to the policy as epsilon greedy.
        for key in keys:
            if(index_lookup(key) in max_action_index): policy[state][key] = p_max_transitions
            else: policy[state][key] = epsilon
    return(policy)                

In [15]:
Q_policy = update_policy(initial_policy, Q, 0.01)    
Q_policy

{0: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
 1: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 2: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 3: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 4: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 5: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 6: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 7: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 8: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 9: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 10: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 11: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 12: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 13: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 14: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 15: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01}}

> **Exercise 9-1-3:** The question is now, how much of an improvement does this policy represent? To find out, do the following:   
> 1. In the cell bellow create and execute the code to compute the state values for the improved policy. Print the sum of the state values along with the state value array.      
> 2. Examine the state values. Do these state values represent an improvement over the initial policy and why?   

In [16]:
state_values = td_0_state_values(Q_policy, 20000).reshape((4,4))     
print(np.sum(state_values))
print(state_values)  

146.30976176959035
[[0.         9.99802773 9.8892897  9.77951069]
 [9.99898129 9.89378226 9.78376786 9.67622067]
 [9.88394403 9.78200452 9.67743171 9.56063727]
 [9.69434054 9.67661522 9.56257872 9.45262955]]


> **Answer:**   
It seems to be an improvement. The new policy incentivize the following:
- 15 - Left
- 14, 13, 12 - Up
- 11 - Left
- 10,8,7 - Up
- 7,6,5 - Left
- 4 - Up
- 3,2,1 - Left

> At first glance this is a good policy and it seems the model is spending the less amount of time in the far away states by looking at the state values and also at the sum of all state values (positive number).

## On Policy vs. Off Policy Algorithms

In this lesson we will explore examples of two broad categories of RL algorithms known as **on policy** and **off policy** methods. 

On policy methods evaluate and improve a single policy. On policy methods converge quickly and often to good solution. In general, **exploration** is performed using $\epsilon$-greedy methods. The TD(0) and MC algorithms we have examined are examples of on policy methods. On policy algorithms are known to have good convergence properties. 

In contrast, off policy methods use two policies. The policy the agent is following is called the **behavior policy**, denoted $b(A_t | S_t)$. The policy being improved is known as the **target policy**, denoted $\pi (A_t | S_t)$. The agent obtains samples of the environment while following the behavior policy. These samples are used to improve the target policy. An advantage of off policy methods is that a deterministic behavior policy can be used while a better target policy is developed. 

## Q-Learning

As we have just seen, the SARSA algorithm is an on policy action value TD estimation method. The **Q-learning** algorithm is a **off policy** TD action value estimation method. 

The update formula for single step Q-learning or **Q-learning(0)** is:

$$Q(S_t,A_t) = Q(S_t,A_t) + \alpha \big[ R_{t+1} + \gamma\ max_a Q(S_{t+1},a) - Q(S_t,A_t) \big]$$  

Where,   
$\delta_t = R_{t+1} + \gamma max_a Q(S_{t+1},a) - Q(S_t,A_t) = $ the TD error,   
$max_a = $ the maximum operator applied to all possible actions in state $S_{t+1}$,   
$Q(S_t,A_t) = $ is the action value in state S given action A,  
$R_{t+1} = $ is the reward for the next time step,   
$\alpha = $ the learning rate,   
$\gamma = $ discount factor.  

The use of the operator $max_a$ makes Q-learning greedy. But, why does using this operator result in an off-policy algorithm? To answer this question, examine the backup diagram shown below. 

<img src="img/Q-Learning.JPG" alt="Drawing" style="width:200px; height:150px"/>
<center> **Backup Diagram for one-step Q-Learning** </center>

The $max_a$ greedily picks the action with the greatest value, regardless of policy. Therefore, Q-learning is an off-policy algorithm. 

### Q-Learning Example

> **Exercise 9-1-3:** The code in the cell below implements the one step Q-learning(0) algorithm. The code is similar to the SARSA(0) code shown previously. The main difference is the addition of the $max_a$ operation when computing the TD error, $\delta_t$.  Additional details on this algorithm can be seen by reading the code comments.   
 > Complete the line of code in the `update_Q` function below to compute the new action value using the single step bootstrap relationship from the formula shown above Execute your code for the random walk policy on the grid world and examine the results. 
 

In [17]:
def simulate_environment_Q(s, action, neighbors = neighbors, rewards = rewards, terminal = 0):
    """
    Function simulates the environment for Q-learning.
    returns s_prime and reward given s and action
    """
    s_prime = neighbors[s][action]
    reward_prime = np.array([rewards[s_prime][a] for a in rewards[0].keys()])
    return (s_prime, reward_prime, is_terminal(s_prime, terminal))

def start_Q_episode(n_states, n_actions):
    '''Function to find a random starting values for the episode
    that is not the terminal state'''
    state = nr.choice(range(n_states))
    while(is_terminal(state)):  ## Make sure not starting at the terminal state
         state = nr.choice(range(n_states))
    ## Now find a random starting action index
    a_index = nr.choice(range(4), size = 1)[0]
    s_prime, reward, terminal = simulate_environment_Q(state, action_lookup(a_index))   
    return state, a_index, reward[a_index] ## action_lookup(a_index), reward[a_index]

def update_Q(Q, current_state, a_index, reward, alpha, gamma):
    """Function to update the actions values in the Q matrix"""
    ## Get s_prime given s and a
    s_prime, reward_prime, terminal = simulate_environment_Q(current_state, action_lookup(a_index))
    a_prime_index = nr.choice(np.where(reward_prime == max(reward_prime))[0], size = 1)[0]
    
    ######### Complete the code below ######################################
    ## Update the action values using the current state and action and the successor state and action (index) 
    ## for the maximum acton value
    Q[current_state,a_index] = Q[current_state,a_index] + alpha * (reward + gamma * (Q[s_prime,a_prime_index] - Q[current_state,a_index]))
     
    return Q, s_prime, reward_prime, terminal, a_prime_index

def Q_learning_0(policy, episodes, alpha = 0.02, gamma = 0.9):
    """
    Function to perform Q-learning(0) control policy improvement.
    """
    ## Initialize the state list and action values
    states = list(policy.keys())
    n_states = len(states)
    n_actions = len(policy[0].keys())
    
    ## Initialize Q matrix
    Q = np.zeros((n_states,n_actions))
    
    for _ in range(episodes): # Loop over the episodes
        terminal = False
        ## Find the inital state, action index and reward
        current_state, a_index, reward = start_Q_episode(n_states,n_actions)
        
        while(not terminal): # Episode ends where get to terminal state   
            ## Update the action values in Q
            Q, s_prime, reward_prime, terminal, a_prime_index = update_Q(Q, current_state, a_index, reward, alpha, gamma)
            ## Set action, reward and state for next iteration
            a_index = a_prime_index
            current_state = s_prime
            reward = reward_prime[a_prime_index]
    return(Q)

In [18]:
Q = Q_learning_0(initial_policy, 10000)
print_Q(Q)

           up      down       left     right
0    0.000000  0.000000   0.000000  0.000000
1    9.623214  9.055078  11.111111  8.826964
2    7.813655  9.184547  11.265402  9.204071
3    6.775910  8.895274   9.876621  7.533558
4   11.111111  9.318104   9.470999  8.961904
5   10.936872  9.202550  10.913823  9.093790
6    9.793753  8.718597   9.881251  8.795395
7    9.163309  8.533632   9.237913  6.407003
8   11.623302  9.254313   7.882688  9.235585
9    9.852146  8.866984  10.045187  8.734356
10   9.261510  8.548595   9.235456  8.449266
11   8.803563  8.385616   8.767207  6.302545
12   9.984957  7.260905   7.141156  8.893033
13   9.280190  6.735450   9.293043  8.599584
14   8.736877  6.591960   8.916203  8.406994
15   8.509766  6.432195   8.582101  6.064432


In [19]:
Q_policy = update_policy(initial_policy, Q, 0.01)    
Q_policy

{0: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
 1: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 2: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 3: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 4: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 5: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 6: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 7: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 8: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 9: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 10: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 11: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 12: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 13: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 14: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 15: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01}}

> **Exercise 9-1-5:** The question is now, how much of an improvement does this policy represent? To find out, execute the code to compute the state values for the improved policy. 

> Examine the state values and provide short answers to these questions:    
> 1. Do these state values represent an improvement over the initial policy and why?   
> 2. Notice that the Q-Learning algorithm was run for half as many iterations as the SARSA algorithm. How does the computational performance of these two algorithms appear to compare for this example?     

In [20]:
state_values = td_0_state_values(Q_policy, 20000).reshape((4,4))     
print(np.sum(state_values))
print(state_values)

146.19763876598742
[[0.         9.9800351  9.88221993 9.7632576 ]
 [9.99841257 9.88159716 9.7842445  9.68214337]
 [9.85683267 9.77286059 9.66621328 9.55991406]
 [9.75669197 9.65453081 9.53598124 9.42270392]]


> **Answers:**   
> 1. Yes, compared to the initial policy this is an improved policy, which we can see by looking at the sum of state values (positive number). 

- 15,14,12 - Left
- 12,11,10 - Up
- 9 - Left
- 8 - Up
- 7,6 - Left
- 5,4 - Up
- 3,2,1 - Left

> 2. For this particular example it seems the SARSA algorithm is better as it gets to a similar policy and final state values compared to the Q-Learning algorithm but half the time (iterations).      

### Double Q-Learning

The Q-learning algorithm just presented has a significant **bias**. To understand why this might be consider the following thought experiment. In most cases the sampled action values are inaccurate. Some action values will have a positive error and some will have a negative error. In the error is on the order of the values themselves, the $max_a$ operator has a reasonable chance of selecting an action value that is the largest because of this error. However, the $max_a$ operator will never select an action with a low value simply because of the errors. The net result is a bias toward action values with the largest positive error. 

What can be done to correct this situation? One relatively simple and effective algorithm is known as **double Q-learning**. Double Q-learning maintains two tables of action values. The values from one table are used to perform the bootstrap updates of the other table and vice versa. This approach averages out the bias. For two tables, $Q_1$ and $Q_2$ we can express double Q-learning as follows:

$$Q_1(S_t,A_t) = Q_1(S_t,A_t) + \alpha \big[ R_{t+1} + \gamma Q_2 \big( S_{t+1},argmax_a Q_1(S_{t+1} , a) \big) - Q_1(S_t,A_t) \big] \\
Q_2(S_t,A_t) = Q_2(S_t,A_t) + \alpha \big[ R_{t+1} + \gamma Q_1 \big( S_{t+1},argmax_a Q_2(S_{t+1} , a) \big) - Q_2(S_t,A_t) \big]$$  

With a 0.5 probability one or the other of these expressions is used for the TD update at each time step. While double Q-learning requires twice as much memory, to maintain the two tables, the computational complexity is the same when compared to Q-learning. 

> **Note:** Another **unbiased** one step off policy TD algorithm is known as **expected SARSA**. See Section 6.6 of Sutton and Barto, second edition, for details.   

### Example of Double Q-Learning

The code in the cell below implements the double Q-learning algorithm. The steps are essentially the same as the foregoing code, except for the updates of the two Q tables. Additional details on this algorithm can be seen by reading the code comments.  

Execute this code and examine the results.

In [21]:
def update_double_Q(q1, q2, current_state, a_index, reward, alpha, gamma):
    """Function to update the actions values in the Q matrix"""
    ## Get s_prime given s and a
    s_prime, reward_prime, terminal = simulate_environment_Q(current_state, action_lookup(a_index))
    a_prime_index = nr.choice(np.where(reward_prime == max(reward_prime))[0], size = 1)[0]
    ## Update the action values 
    q1[current_state,a_index] = q1[current_state,a_index] + alpha * (reward + gamma * (q2[s_prime,a_prime_index] - q1[current_state,a_index]))
    return q1, s_prime, reward_prime, terminal, a_prime_index

def double_Q_learning_0(policy, episodes, alpha = 0.02, gamma = 0.9):
    """
    Function to perform Q-learning(0) control policy improvement.
    """
    ## Initialize the state list and action values
    states = list(policy.keys())
    n_states = len(states)
    n_actions = len(policy[0].keys())
    
    ## Initialize both Q matricies
    Q1 = np.zeros((n_states,n_actions))
    Q2 = np.zeros((n_states,n_actions))
    
    for _ in range(episodes): # Loop over the episodes
        terminal = False
        ## Find the inital state, action index and reward
        current_state, a_index, reward = start_Q_episode(n_states,n_actions)
        
        while(not terminal): # Episode ends where get to terminal state   
            ## Update the action values in Q1 or Q2 based on random choice
            if(nr.uniform() <= 0.5):
                Q1, s_prime, reward_prime, terminal, a_prime_index = update_double_Q(Q1, Q2, current_state, a_index, reward, alpha, gamma)
            else:
                Q2, s_prime, reward_prime, terminal, a_prime_index = update_double_Q(Q2, Q1, current_state, a_index, reward, alpha, gamma)
            ## Set action, reward and state for next iteration
            a_index = a_prime_index
            current_state = s_prime
            reward = reward_prime[a_prime_index]
    return(Q1)

In [22]:
Q = double_Q_learning_0(initial_policy, 10000)
print_Q(Q)

           up      down       left     right
0    0.000000  0.000000   0.000000  0.000000
1    7.260859  5.895400  11.111111  6.337240
2    4.930421  7.998711  11.474780  7.963728
3    3.676620  7.499623   9.137741  4.314966
4   11.111111  5.561853   7.851174  5.569974
5   10.907200  8.044138  10.889236  8.009072
6    9.064280  7.308209   9.337534  7.496114
7    8.034120  7.143178   8.047169  3.561742
8   11.368858  7.931791   5.850591  7.989397
9    9.225082  7.502528   8.944598  7.299339
10   8.139971  7.057371   8.267702  7.018490
11   7.544746  6.751191   7.426521  3.014642
12   9.061741  3.802020   4.433444  7.533121
13   8.010643  3.437493   8.009496  7.073749
14   7.325070  3.301901   7.451323  6.769249
15   7.153266  2.667910   7.087774  2.854863


With the Q values computed, execute the code in the cell below to find an improved policy.  

In [23]:
double_Q_policy = update_policy(initial_policy, Q, 0.01)    
double_Q_policy

{0: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
 1: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 2: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 3: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 4: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 5: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 6: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 7: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 8: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 9: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 10: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 11: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 12: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 13: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 14: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 15: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01}}

> **Exercise 9-1-5:** The question is now, how much of an improvement does this policy represent and how does it compare to the other algorithms? To find out, execute the code to compute the state values for the improved policy.    

In [24]:
state_values = td_0_state_values(Q_policy, 20000).reshape((4,4))     
print(np.sum(state_values))
print(state_values)  

146.19770141464662
[[0.         9.98446115 9.86452377 9.69041912]
 [9.99267819 9.87918341 9.77344496 9.67554892]
 [9.88212038 9.77211461 9.66700084 9.57680543]
 [9.74371558 9.66739347 9.55886451 9.46942709]]


> Examine these results. Keep in mind that the point of double Q-Learning is reduced bias, which will be minimal with a small number of possible state-action values. What evidence can you see for improvement when these results are compared to the original Q-Learning results?      

> **Answer:**     

#### Copyright 2018, 2019, 2022, Stephen F Elston. All rights reserved. 